In [1]:
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.2/387.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 68.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.2/201.2 kB 27.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 2

In [2]:
from flair.data import Sentence
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.datasets import ColumnCorpus
from flair.datasets import SentenceDataset
from flair.data import Token
from tqdm import tqdm
import gensim
from gensim.models.word2vec import Word2Vec
import pandas as pd
pd.set_option('max.colwidth', 0)

# Prepare Dataset

In [3]:
dataset_file = "mypos-ver.3.0-flair.txt"

# Define the columns in dataset
columns = {0: 'text', 1: 'pos'}

# Initialize the corpus
corpus = ColumnCorpus(data_folder='.', column_format=columns, train_file=dataset_file)

2023-11-29 05:26:07,201 Reading data from .
2023-11-29 05:26:07,206 Train: mypos-ver.3.0-flair.txt
2023-11-29 05:26:07,208 Dev: None
2023-11-29 05:26:07,209 Test: None
2023-11-29 05:26:19,071 No test split found. Using 0% (i.e. 4320 samples) of the train split as test data
2023-11-29 05:26:19,102 No dev split found. Using 0% (i.e. 3888 samples) of the train split as dev data


# Load pre-trained FlairEmbeddings model



In [4]:
from flair.embeddings import FlairEmbeddings, StackedEmbeddings, WordEmbeddings
flair_forward_embedding = FlairEmbeddings('best-lm-fw.pt')
flair_backward_embedding = FlairEmbeddings('best-lm-bw.pt')

In [5]:
# Stack the embeddings
embedding_types = [

    FlairEmbeddings('best-lm-fw.pt'),
    FlairEmbeddings('best-lm-bw.pt')
]

embeddings = StackedEmbeddings(embeddings=embedding_types)

In [6]:
# test stacked embedding
sentence = Sentence('ဒါ ပေါ့ ။ ကိစ္စ မ ရှိ ပါ ဘူး ။')
embeddings.embed(sentence)

for token in sentence:
    print(token)
    print(token.embedding)

Token[0]: "ဒါ"
tensor([-3.7974e-01,  2.1368e-05,  4.7706e-04,  7.7587e-04, -2.9287e-03,
         4.2294e-02, -7.3442e-04, -5.4222e-04,  7.6816e-03,  9.7218e-04,
        -2.0761e-02,  4.2603e-02, -2.0649e-04,  1.3703e-01, -7.4877e-01,
        -1.1288e-02,  3.6933e-02,  1.6755e-02, -4.5331e-03,  1.4367e-04,
         2.4181e-01, -3.2227e-02,  5.6861e-02,  2.0548e-01, -6.6127e-02,
         1.8289e-03,  3.1663e-02, -5.9485e-05, -2.4779e-02, -6.3073e-02,
        -1.3252e-03,  8.0238e-02,  1.2424e-03,  1.7425e-02, -2.8310e-02,
         2.3591e-02,  4.0888e-03, -3.2991e-03,  9.5268e-03,  5.7763e-01,
        -1.5888e-02, -2.5222e-01, -2.3787e-01,  3.0005e-01,  5.3408e-03,
        -1.8336e-01,  2.2487e-04,  2.1046e-03,  1.5220e-02, -2.8578e-04,
         1.7536e-03, -4.2489e-03,  7.3850e-04, -1.4167e-02,  4.7915e-01,
         6.5196e-02,  5.6772e-01, -7.6392e-03,  8.3006e-01, -5.1166e-02,
         8.1143e-02,  4.9698e-01, -4.7595e-05, -7.5586e-02,  5.3671e-01,
         2.5885e-02, -1.1890e-01,  7

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

sentence_1 = Sentence('ဒီ လို ပဲ ပေါ့ ကိစ္စ မ ရှိ ပါ ဘူး')
sentence_2 = Sentence('ဒါ ပေါ့ ကိစ္စ မ ရှိ ပါ ဘူး')

embeddings.embed([sentence_1, sentence_2])

for i1 in sentence_1:
    print(i1.embedding)

for i2 in sentence_2:
    print(i2.embedding)

embedding_1 = i1.embedding.unsqueeze(0).cpu().numpy()
embedding_2 = i2.embedding.unsqueeze(0).cpu().numpy()

cosine_similarity(embedding_1, embedding_2)

tensor([ 2.9254e-02, -6.4189e-05, -8.8054e-03, -1.0163e-01, -1.7472e-03,
         4.8585e-04, -2.2885e-04,  2.7267e-04,  1.1107e-02, -8.9043e-02,
        -1.3916e-02,  5.3967e-03, -1.2641e-04,  6.2256e-02, -7.3497e-01,
         5.1309e-02,  7.0814e-02,  1.7537e-02, -4.2601e-02, -3.8446e-06,
         2.6575e-01, -1.4385e-03, -1.0783e-01,  9.9054e-02, -4.3456e-02,
         1.9507e-03, -3.2171e-03, -7.6641e-07,  3.2656e-04,  8.8832e-02,
        -1.8836e-03,  6.6630e-04,  6.5986e-04,  2.5487e-02, -1.6228e-01,
         2.2297e-02, -3.1839e-03, -5.1704e-01,  1.0487e-02,  8.4584e-02,
        -1.0984e-04, -3.0345e-02,  3.5269e-02,  6.3918e-02,  1.4744e-01,
         6.0292e-02,  1.1620e-04,  1.6999e-03,  1.9981e-03, -2.6658e-04,
         2.3658e-04, -2.6566e-01, -3.5826e-05, -3.9221e-02,  2.1956e-01,
        -2.0117e-02, -3.2813e-01, -2.6102e-03,  7.4787e-01, -3.8880e-01,
        -7.5354e-01,  4.8404e-01, -6.5893e-05, -7.0573e-01, -7.3216e-02,
        -8.6228e-04, -6.8223e-02,  7.6088e-01, -1.4

array([[0.999768]], dtype=float32)

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

sentence_1 = Sentence('သူ တို့ က သူငယ်ချင်း တွေ')
sentence_2 = Sentence('ဒါ ပေါ့ ကိစ္စ မ ရှိ ပါ ဘူး')

embeddings.embed([sentence_1, sentence_2])

for i1 in sentence_1:
    print(i1.embedding)

for i2 in sentence_2:
    print(i2.embedding)

embedding_1 = i1.embedding.unsqueeze(0).cpu().numpy()
embedding_2 = i2.embedding.unsqueeze(0).cpu().numpy()

cosine_similarity(embedding_1, embedding_2)

tensor([ 1.7913e-02,  6.2571e-03, -2.4316e-02,  2.2651e-01, -1.9427e-01,
        -4.2907e-01, -2.7153e-03,  4.6092e-05, -2.8229e-02,  5.6901e-03,
        -1.3612e-02,  7.5855e-02,  8.3727e-06,  1.4217e-02, -7.0944e-01,
        -5.4399e-03, -1.0358e-01,  3.5143e-02,  8.9646e-03,  5.2612e-02,
         2.4288e-01, -6.0744e-04, -4.7541e-04, -2.0992e-02, -1.6508e-01,
         3.5462e-02,  2.4394e-02,  2.1493e-04,  8.2009e-05,  6.3666e-01,
        -1.8794e-01,  1.3086e-02,  2.0589e-03,  3.4556e-03, -5.1886e-03,
         2.3821e-01,  4.4327e-02, -1.0760e-02,  1.6948e-02,  6.6744e-01,
        -5.0410e-03, -1.9614e-02, -3.6799e-01,  4.3213e-01, -4.8506e-02,
        -1.8792e-01, -3.3974e-05,  1.8042e-04,  1.3670e-04, -1.6820e-01,
         2.1544e-04, -9.5751e-02,  3.9907e-04, -3.0664e-03,  9.4965e-02,
        -1.2144e-01, -1.8456e-02, -5.4789e-03,  4.7942e-02, -2.3379e-01,
         1.9085e-01,  2.6867e-03,  3.5216e-05,  7.0800e-02,  6.0919e-03,
        -2.8406e-01, -7.8888e-01,  7.8967e-01,  1.3

array([[0.1703175]], dtype=float32)

# Sequence Tagging

In [9]:
label_type = 'pos'

# Create a label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type, add_unk=True)

# Create model
model = SequenceTagger(hidden_size=256,
                      embeddings=embeddings,
                      tag_dictionary=label_dict,
                      tag_type=label_type)

# Create the trainer and train the model
trainer = ModelTrainer(model, corpus)
trainer.train('pos_tagger', learning_rate=0.1, mini_batch_size=32, max_epochs=10)

2023-11-29 05:26:37,605 Computing label dictionary. Progress:


0it [00:00, ?it/s]
34989it [00:01, 22056.41it/s]

2023-11-29 05:26:39,234 Dictionary created for label 'pos' with 17 values: part (seen 108167 times), n (seen 85233 times), ppm (seen 70276 times), v (seen 62551 times), punc (seen 43844 times), pron (seen 16425 times), conj (seen 14449 times), adj (seen 12826 times), adv (seen 8706 times), num (seen 4789 times), tn (seen 4662 times), fw (seen 2655 times), int (seen 530 times), abb (seen 289 times), sb (seen 212 times), O (seen 1 times)
2023-11-29 05:26:39,237 SequenceTagger predicts: Dictionary with 17 tags: <unk>, part, n, ppm, v, punc, pron, conj, adj, adv, num, tn, fw, int, abb, sb, O
2023-11-29 05:26:39,255 ----------------------------------------------------------------------------------------------------
2023-11-29 05:26:39,256 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.1, inplace=False)
        (encoder): Embedding(173, 100)
        (rnn): LSTM(100, 128)
      )
    )


2023-11-29 05:26:39,287 ----------------------------------------------------------------------------------------------------
2023-11-29 05:26:39,289 Final evaluation on model from best epoch (best-model.pt)
2023-11-29 05:26:39,290  - metric: "('micro avg', 'f1-score')"
2023-11-29 05:26:39,291 ----------------------------------------------------------------------------------------------------
2023-11-29 05:26:39,292 Computation:
2023-11-29 05:26:39,293  - compute on device: cuda:0
2023-11-29 05:26:39,294  - embedding storage: cpu
2023-11-29 05:26:39,295 ----------------------------------------------------------------------------------------------------
2023-11-29 05:26:39,297 Model training base path: "pos_tagger"
2023-11-29 05:26:39,298 ----------------------------------------------------------------------------------------------------
2023-11-29 05:26:39,299 ----------------------------------------------------------------------------------------------------
2023-11-29 05:26:49,585 epo

100%|██████████| 61/61 [00:16<00:00,  3.70it/s]


...

100%|██████████| 61/61 [00:13<00:00,  4.46it/s]


2023-11-29 05:42:55,599 DEV : loss 0.1849522590637207 - f1-score (micro avg)  0.9433
2023-11-29 05:42:55,791  - 0 epochs without improvement
2023-11-29 05:42:55,792 saving best model
2023-11-29 05:42:55,826 ----------------------------------------------------------------------------------------------------
2023-11-29 05:42:55,832 Loading model from best epoch ...
2023-11-29 05:42:55,849 SequenceTagger predicts: Dictionary with 19 tags: <unk>, part, n, ppm, v, punc, pron, conj, adj, adv, num, tn, fw, int, abb, sb, O, <START>, <STOP>


100%|██████████| 68/68 [00:09<00:00,  6.86it/s]


2023-11-29 05:43:06,314 
Results:
- F-score (micro) 0.9415
- F-score (macro) 0.8643
- Accuracy 0.9415

By class:
              precision    recall  f1-score   support

        part     0.9701    0.9408    0.9552     13532
           n     0.8943    0.9571    0.9247     10430
         ppm     0.9802    0.9819    0.9810      8616
           v     0.9353    0.9233    0.9293      7655
        punc     0.9985    0.9996    0.9991      5386
        pron     0.9562    0.9576    0.9569      2074
        conj     0.8821    0.9366    0.9086      1814
         adj     0.8173    0.6588    0.7295      1650
         adv     0.7181    0.7287    0.7233      1080
         num     0.9984    0.9952    0.9968       630
          tn     0.9457    0.9294    0.9375       581
          fw     0.9962    1.0000    0.9981       263
         int     0.9074    0.8167    0.8596        60
          sb     1.0000    0.7692    0.8696        39
         abb     1.0000    0.1081    0.1951        37

    accuracy         

{'test_score': 0.9415194904080079}

# Predict

In [10]:
# load the model you trained
model = SequenceTagger.load('pos_tagger/final-model.pt')

sentence = Sentence('အသစ် ဝယ် ထား တဲ့ ဆွယ်တာ က အသီးထ ပါ တယ်')

model.predict(sentence)
print(sentence.to_tagged_string())

2023-11-29 05:58:27,562 SequenceTagger predicts: Dictionary with 19 tags: <unk>, part, n, ppm, v, punc, pron, conj, adj, adv, num, tn, fw, int, abb, sb, O, <START>, <STOP>
Sentence[9]: "အသစ် ဝယ် ထား တဲ့ ဆွယ်တာ က အသီးထ ပါ တယ်" → ["အသစ်"/n, "ဝယ်"/v, "ထား"/part, "တဲ့"/part, "ဆွယ်တာ"/n, "က"/ppm, "အသီးထ"/v, "ပါ"/part, "တယ်"/ppm]
